# Import a survey file from S3

In [1]:
!pip install --upgrade pip boto3 citric --quiet

In [2]:
import io
import logging

import boto3
from IPython.display import HTML

import citric

## Set up logging

In [3]:
formatter = logging.Formatter("{asctime} {levelname} {message}", style="{")

handler = logging.StreamHandler()
handler.setFormatter(formatter)

logger = logging.getLogger("citric")
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

## Move survey file to S3

- Setup S3 client
- Create a new bucket
- Upload file from local path

In [4]:
!ls ../../examples

free_text.lsq                limesurvey_survey_432535.lss
group.lsg                    survey.lss


In [5]:
s3 = boto3.client("s3")

# use your own bucket name here
s3.create_bucket(Bucket="testing")
s3.upload_file("../../examples/survey.lss", "testing", "survey.lss")

## Upload survey from S3

- Start `citric` client
- Download file from S3
- Import survey from S3 file
- Display survey questions

In [6]:
# Use your own server's parameters here
with citric.Client(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
) as client:
    file_object = io.BytesIO()
    s3.download_fileobj("testing", "survey.lss", file_object)

    file_object.seek(0)
    survey_id = client.import_survey(file_object)
    questions = client.list_questions(survey_id)

    for question in questions:
        display(HTML(question["question"] + question["help"] + "<hr>"))

2024-02-08 11:25:12,778 INFO Invoked RPC method get_session_key with ID 119194
2024-02-08 11:25:13,035 INFO Invoked RPC method import_survey with ID 878109
2024-02-08 11:25:13,108 INFO Invoked RPC method list_questions with ID 942978


2024-02-08 11:25:13,161 INFO Invoked RPC method release_session_key with ID 948663


## Cleanup

- Delete bucket
- Delete survey

In [7]:
s3.delete_object(Bucket="testing", Key="survey.lss")
s3.delete_bucket(Bucket="testing")

with citric.Client(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
) as client:
    client.delete_survey(survey_id)

2024-02-08 11:25:13,291 INFO Invoked RPC method get_session_key with ID 500759
2024-02-08 11:25:13,379 INFO Invoked RPC method delete_survey with ID 409998
2024-02-08 11:25:13,430 INFO Invoked RPC method release_session_key with ID 315195
